In [8]:
import collections
import attr
import functools
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
from matplotlib import pyplot
from keras.models import Sequential
from math import sqrt 
import numpy as np 
import scipy.misc 
from IPython.display import display 
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator 
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization,Activation,Average
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools

import nest_asyncio
nest_asyncio.apply()

np.random.seed(0)




In [9]:
import pandas as pd
import numpy as np
df_n = pd.read_csv("out.csv")
df_o = pd.read_csv("new.csv")
data = pd.read_csv("dataset.csv")
data = data.iloc[:,1:]

emotion = df_o.iloc[:,4]
df = pd.concat([data, emotion], ignore_index=True, axis=1)

TRAIN_DATA_PER_CLIENT = 50

TRAINING_EPOCHS = 3


x_client_1 = data[0 : TRAIN_DATA_PER_CLIENT]
y_client_1 = emotion[0 : TRAIN_DATA_PER_CLIENT]


In [34]:

from tensorflow import reshape
from collections import OrderedDict

def create_federated_data(x_train, y_train):

    orderDict = OrderedDict()

    orderDict['x'] = np.array(x_train)
    orderDict['y'] = np.array(y_train)
    dataset = tf.data.Dataset.from_tensor_slices(orderDict)
    batch = dataset.shuffle(10).batch(5)

    return batch

federated_data_client_1 = [create_federated_data(x_client_1, y_client_1) for epoch in range(TRAINING_EPOCHS)]

federated_data_client_1[1]

<BatchDataset shapes: OrderedDict([(x, (None, 9)), (y, (None,))]), types: OrderedDict([(x, tf.float64), (y, tf.int64)])>

In [11]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy


def create_model_lite():
    input_shape=(48,48,1)

    model = Sequential()
    model.add(Conv2D(6, (5, 5), input_shape=input_shape, padding='same', activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(16, (5, 5), padding='same', activation = 'relu'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation = 'relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    
    
    
    return model

def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(64,input_shape=(9,), activation='relu'))
	
	
	
	return model

In [12]:
model1 = create_model_lite()
print(model1.summary())
model2 = baseline_model()
print(model2.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 6)         156       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 16)        2416      
_________________________________________________________________
activation (Activation)      (None, 24, 24, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0

In [13]:
from keras.layers import add
merged_output = add([model1.output, model2.output])   
model_combined = Sequential()
model_combined.add(Activation('relu'))
model_combined.add(Dense(128))
model_combined.add(Activation('relu'))
model_combined.add(Dense(7, activation='softmax'))

print(model1.input,model2.input)

final_model = Model([model1.input, model2.input], model_combined(merged_output))

# final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(final_model.summary())



KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 1), dtype=tf.float32, name='conv2d_input'), name='conv2d_input', description="created by layer 'conv2d_input'") KerasTensor(type_spec=TensorSpec(shape=(None, 9), dtype=tf.float32, name='dense_1_input'), name='dense_1_input', description="created by layer 'dense_1_input'")
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
conv2d_input (InputLayer)       [(None, 48, 48, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 6)    156         conv2d_input[0][0]               
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 24, 24, 6)    0          

In [22]:
# dummy_batch = tf.nest.map_structure(lambda x: x.numpy(), iter(federated_data_client_1[0]).next())
# print(dummy_batch['x'].shape)

def model_fn():
  keras_model = baseline_model()
  return tff.learning.from_keras_model(
      keras_model,
      input_spec=federated_data_client_1[0].element_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [23]:
model = model_fn()
model


In [17]:
#model.weights.trainable

# A server-to-client broadcast step. 

# A local client update step. 

# A client-to-server upload step. 

# A server update step. 



In [24]:
iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02))

str(iterative_process.initialize.type_signature)


'( -> <model=<trainable=<float32[9,64],float32[64]>,non_trainable=<>>,optimizer_state=<int64>,delta_aggregate_state=<value_sum_process=<>,weight_sum_process=<>>,model_broadcast_state=<>>@SERVER)'

In [25]:
state = iterative_process.initialize()

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [27]:
state, metrics = iterative_process.next(state, federated_data_client_1)
print('round  1, metrics={}'.format(metrics))

round  1, metrics=OrderedDict([('broadcast', ()), ('aggregation', OrderedDict([('mean_value', ()), ('mean_weight', ())])), ('train', OrderedDict([('sparse_categorical_accuracy', 0.006666667), ('loss', 12.999751)])), ('stat', OrderedDict([('num_examples', 150)]))])
